In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import pandas as pd

In [3]:
### Generating our data

In [4]:
u_x = stats.norm(scale = 2)
u_y = stats.norm(scale=2)
z = stats.norm(scale=2)
u = stats.truncnorm(0, np.infty, scale=4)

# exogenous
n = 5000
u_x = u_x.rvs(n)
u_y = u_y.rvs(n)

# endogenous
u = u.rvs(n)
z = z.rvs(n)
x = -0.2*u + 0.7*z + 0.1*u_x
y = 0.7*u + 0.35*x + 0.1*u_y

### Y = f(Z)

Z -> X <- U -> Y is blocked since X is a collider

Hence only open path Z -> X -> Y

In [5]:
## Fitting our models
X = pd.DataFrame({'Z':z})
X = sm.add_constant(X, prepend=True)
model = sm.OLS(y,X)
results_yz = model.fit()
print(results_yz.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.098
Method:                 Least Squares   F-statistic:                     545.7
Date:                Fri, 07 Jun 2024   Prob (F-statistic):          1.23e-114
Time:                        16:49:43   Log-Likelihood:                -9300.4
No. Observations:                5000   AIC:                         1.860e+04
Df Residuals:                    4998   BIC:                         1.862e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0574      0.022     93.567      0.0

In [6]:
# prompt: print parameters of the model

print(results_yz.params)

const    2.057433
Z        0.258311
dtype: float64


In [7]:
print(results_yz.params['Z'])

0.25831087068641323


### X = f(Z)

between X and Z again there is just one causal path and no spurious association

In [8]:
## Fitting our models
X = pd.DataFrame({'Z':z})
X = sm.add_constant(X, prepend=True)
model = sm.OLS(x,X)
results_xz = model.fit()
print(results_xz.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.873
Method:                 Least Squares   F-statistic:                 3.424e+04
Date:                Fri, 07 Jun 2024   Prob (F-statistic):               0.00
Time:                        16:49:48   Log-Likelihood:                -3913.6
No. Observations:                5000   AIC:                             7831.
Df Residuals:                    4998   BIC:                             7844.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6519      0.007    -87.074      0.0

In [9]:
print(results_xz.params['Z'])

0.696658343311734


## Causal effect of X -> Z

In [10]:
results_yz.params['Z']/results_xz.params['Z']

0.37078558401880324

Which is very close to the true causal impact of X on Y